In [ ]:
from util import kinect as util_kine
import math
import time
from pykinect_azure import Transformation
from pykinect_azure import K4A_CALIBRATION_TYPE_COLOR, K4A_CALIBRATION_TYPE_DEPTH, k4a_float2_t
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
kinect = util_kine.Kinect()

In [ ]:
def quat2eulers(q0:float, q1:float, q2:float, q3:float) -> tuple:
    roll = math.atan2(
        2 * ((q2 * q3) + (q0 * q1)),
        q0**2 - q1**2 - q2**2 + q3**2
    )  # radians
    pitch = math.asin(2 * ((q1 * q3) - (q0 * q2)))
    yaw = math.atan2(
        2 * ((q1 * q2) + (q0 * q3)),
        q0**2 + q1**2 - q2**2 - q3**2
    )
    return (roll, pitch, yaw)

class Observer:
  def __init__(self, kinect=None):
    if kinect is None:
      self.kinect = util_kine.Kinect()
    else:
      self.kinect = kinect
    
    self.info=None
    
  def update(self):
    self.kinect.update()
    neck, pelvis = self.kinect.neck, self.kinect.pelvis
    
    neck_pos, neck_ori = neck[:3], neck[3:7]
    pelvis_pos, pelvis_ori = pelvis[:3], pelvis[3:7]
    
    spine_vec = neck_pos - pelvis_pos
    self.slope = spine_vec[1]/spine_vec[0]
    yaw_pelvis = quat2eulers(*pelvis_ori)[0]
    yaw_neck = quat2eulers(*neck_ori)[0]
    
    self.yaw_torso = (yaw_pelvis+yaw_neck)/2
  
obs = Observer(kinect)


In [ ]:
obs.update()

joints2d = obs.kinect.joints2d()
roll, pitch, yaw = quat2eulers(*obs.kinect.neck[3:7])

print(obs.kinect.neck)
print(f"""
position: {obs.kinect.neck[:3]}
position_2d: {joints2d["neck"]}
orientation: {obs.kinect.neck[3:7]}
roll, pitch, yaw: {roll, pitch, yaw}
confidence_level: {obs.kinect.neck[-1]}
""")


In [ ]:
obs.update()
depth_width, depth_height = obs.kinect.depth_img.shape
XGRID, YGRID = np.meshgrid(np.arange(depth_width), np.arange(depth_height))

gradation_hor = XGRID + obs.slope*YGRID

"""
p_ret, p_cloud = obs.kinect.capture.get_pointcloud()

p_norm = np.linalg.norm(p_cloud, axis=1)
p_cloud=p_cloud[p_norm>0.]
print(p_cloud.shape)
X = p_cloud[:,0]
print(X.min(), X.max())"""

print()
#print(gradation_hor)
#plt.imshow(gradation_hor)

In [ ]:
start = time.time()
while time.time() - start<10:
  obs.update()
  print(f"\r{obs.yaw_torso/math.pi}", end="")